In [8]:
from utilities import load_data
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
X,y = load_data(100, amplitude=10)

In [3]:
X = np.repeat(X, 3, -1)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [10]:
base_model = keras.applications.VGG16(
    weights='imagenet', 
    input_shape=X.shape[1:],
    include_top=False)

In [11]:
base_model.trainable = False

In [12]:
inputs = keras.Input(shape=X.shape[1:])
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

In [13]:
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 400, 400, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 12, 12, 512)       14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________


In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [15]:
history = model.fit(X_train, y_train, epochs=10, batch_size=25, verbose=1)

Epoch 1/10
6/6 [==============================] - 37s 6s/step - loss: 0.9525 - acc: 0.4627
Epoch 2/10
6/6 [==============================] - 41s 7s/step - loss: 0.7204 - acc: 0.4104
Epoch 3/10
6/6 [==============================] - 40s 7s/step - loss: 0.6741 - acc: 0.5373
Epoch 4/10
6/6 [==============================] - 41s 7s/step - loss: 0.6754 - acc: 0.5373
Epoch 5/10
6/6 [==============================] - 41s 7s/step - loss: 0.6739 - acc: 0.5373
Epoch 6/10
6/6 [==============================] - 40s 7s/step - loss: 0.6655 - acc: 0.5373
Epoch 7/10
6/6 [==============================] - 40s 7s/step - loss: 0.6562 - acc: 0.5373
Epoch 8/10
6/6 [==============================] - 40s 7s/step - loss: 0.6514 - acc: 0.5373
Epoch 9/10
6/6 [==============================] - 40s 7s/step - loss: 0.6466 - acc: 0.5373
Epoch 10/10
6/6 [==============================] - 41s 7s/step - loss: 0.6432 - acc: 0.7463


In [17]:
model.evaluate(X_test, y_test, return_dict=True)

3/3 [==============================] - 11s 4s/step - loss: 0.6664 - acc: 0.8485


{'loss': 0.6663767695426941, 'acc': 0.8484848737716675}

In [18]:
model.evaluate(X_train, y_train, return_dict=True)

5/5 [==============================] - 34s 7s/step - loss: 0.6399 - acc: 0.8731


{'loss': 0.6398996710777283, 'acc': 0.8731343150138855}